# Olal: Decentralized cryptocurrency

Bring the control of finance back to people.

## Goals:

Design a programmable crpyotcurrency with the following points.

- Unlike blockchain technology, In Olal, Each account (private public key) has its own books. A receiving ledger book for recording incoming receiving transactions, and a spending ledger for recording spending transactions.
  - Choose the right choice of storage (sqlite3 or virtual file system etc) for storing each acount's key data and ledger books data.
- Transactions in a book is like stack, it can only grow. A transaction happens in the following stages:
  - Sender account initiates a spending transaction from its own spending book (that contains the current total number of transaction of its own recieving and spending ledger) by populating this signed transaction to the entire Olal network (regardless of the transport of connecion, could be websocket through server relay, could be peer to peer connection through similar miniupnpc like bitcoin, or through WebRTC and tunnnelling server). 
  - This sender account's initiation logic is determined by the smart contract written for the receiving account address. The smart contract written in lua will be run by all nodes. This contract code might be run by the sender to initiate a trasnaction, and it also run the the receiving account's node to finalize/confirming the transaction, and it also run by all other nodes on the network to lock ledger states for preventing double spending, or any other logic receiving account impose own its own ledger books.

## Suggestions for Improvement

1. Quantum-Resistant Key Support:
   1. Implementation: Consider integrating a quantum-resistant algorithm like Dilithium. The liboqs library provides implementations of such algorithms.
   2. KeyPair Interface: Extend the KeyPair interface to accommodate additional key types, ensuring methods like sign and verify are appropriately implemented for each algorithm.​
2. Delegate Key Management:
   1. Identification: Implement a method to retrieve delegate keys by their public key or identifier, facilitating management and verification processes.
   2. Revocation: Introduce functionality to revoke or replace delegate keys, enhancing security and flexibility.​
3. Transaction Verification:
   1. Delegate Verification: In the Account class, implement a method to verify if a transaction was signed by a valid delegate key. This ensures that only authorized delegate keys can sign receiving transactions.​
4. Serialization and Deserialization:
   1. Consistency: Ensure that both Account and Transaction classes have robust serialization and deserialization methods. This is crucial for network communication and storage.​
5. Unit Testing:
   1. Coverage: Develop comprehensive unit tests for key functionalities, including key generation, signing, verification, transaction creation, and ledger updates.
   2. Edge Cases: Test scenarios involving invalid signatures, unauthorized delegate keys, and malformed transactions to ensure system robustness.​
6. Documentation:
   1. Usage Examples: Provide examples demonstrating how to create accounts, add delegate keys, create transactions, and verify signatures. This aids in onboarding and understanding the system's capabilities.

### Don't Trust. Verify.

### When user runs the same account on two or more different machines. Multiple pending transactions are observed, how to handle the book keeeping?

Each machine may observe the pending transactions in different order and signed them in different orders, or even on different receiving book's levels.

To solve this problem. Olal transaction uses two handshake mechanism. Sender sends a pending transaction.

- Case 1: same transaction being signed by two differente machines with the same earn_floor, this is OK.
- Case 2: same transaction being signed by two different machiens with different earn_floors: This means one of the machine's earn stack is not synced. The machine with lower earn_floor should ask the network for the conflict trasanction that already occupies the earn_floor
 this transaction might be signed by the receiver on different machines. But sender only picks one for confirmation transaction. Once sender pickes one confirmation transaction, he signed it to form a finalized transaction

A sumr transaction is a 4 phases payment. They are __pending__, __signed__, __finalize__ and __close__.

- Sender first initiates a payment by broadcasting a phase 1 __pending__ transaction to the network. Such broadcast makes the entire network to turn sender's the expense journal to a locked status. Such mechanism prevents sender from spending more than his income journal's value, it also prevents expense journal record inconsistency due to cancelled transactions in lower levels. We'll discuss more details in the "Journal balance".
- Receiver receives the __pending__ transaction from one or more nodes in the network, verifies the transaction data and confirming sender's income journal has enough funding for this transaction. Receiver also tries to get this __pending__ transaction from as many nodes as possible in the network and verify that there's no two or more different __pending__ transactions on the network from the same sender. After verification, receiver signs the __pending__ transaction and broadcast it to the network, thus changing the transaction from __pending__ to __signed__. 

    All nodes can verify __pending__ transactions and mark it as __invalid__ and discard it in the future.

    If nodes are all honest and the __pending__ transaction is valid, receiver should be able to get a copy the the __pending__ transaction from any nodes on the network. However, dishonest nodes could exist. If some nodes are dishonest and they accepts an invalid __pending__ transaction as valid and provided to the receiver. That's all fine, because any node is only-trust-myself node, receiver can verify the __pending__ transaction by itself.

First phase is pending phase initiated by sender and broadcasted to sumr network. The format of the pending phase is $[c, a, v_{o_1}, \hat{a}_1, v_{o_2}, \hat{a}_2, ..., v_{o_c}, \hat{a}_c, v_{oc}, l_i, l_o]$.

- $v_o$ is the sending value


- What if receiver never accepts a payment? How can the sender cancel the pending transaction?
